# Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

## Анализ исходных данных

In [8]:
import pandas as pd
from pymystem3 import Mystem
from collections import Counter

In [3]:
data = pd.read_csv('/datasets/data.csv')
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


**Вывод**
судя по исходным данным есть пропуски в некоторых колонках(days_employed 2174 пропуска), total_income(2174), также второй столбец содержит отрицательные вещественные числа , когда по логике должен иметь положительные целые числи, тоже самое в столбце total_income 

## Предобработка данных

### Обработка пропусков

In [4]:
display(data[['days_employed','total_income']].isna().sum())# определение явных пропусков
data['days_employed'] = data['days_employed'].fillna(data['days_employed'].mean())
data['total_income'] = data['total_income'].fillna(data['total_income'].median())

days_employed    2174
total_income     2174
dtype: int64

Пропущенные значения были обнаружены в столбцах days_employed и total_income
Пропуски были заменены в столбце days_employed на среднее значение по этому столбцу , а в total_income  на медианное значение по столбцу
Возможные причины появления пропусков это ошибка при переносе данных в таблицу, или же человеческий фактор.


### Замена типа данных

In [5]:
data['days_employed'] = data['days_employed'].astype('int')
data['total_income'] = data['total_income'].astype('int')


data['total_income'] = data['total_income'].apply(lambda x : round(x/1000))
data['days_employed'] = data['days_employed'].apply(lambda x : abs(x))
data['days_employed'] = data['days_employed'].apply(lambda x : round(x/365))
data = data.rename(columns={'days_employed':'years_employed'})
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   children          21525 non-null  int64 
 1   years_employed    21525 non-null  int64 
 2   dob_years         21525 non-null  int64 
 3   education         21525 non-null  object
 4   education_id      21525 non-null  int64 
 5   family_status     21525 non-null  object
 6   family_status_id  21525 non-null  int64 
 7   gender            21525 non-null  object
 8   income_type       21525 non-null  object
 9   debt              21525 non-null  int64 
 10  total_income      21525 non-null  int64 
 11  purpose           21525 non-null  object
dtypes: int64(7), object(5)
memory usage: 2.0+ MB


### Обработка дубликатов

In [6]:
data = data.drop_duplicates().reset_index(drop=True) # удаляю явные дубликаты
data['education'] = data['education'].apply(str.lower) # привожу к одному нижнему регистру

print(data['education'].value_counts())

среднее                15188
высшее                  5251
неоконченное высшее      744
начальное                282
ученая степень             6
Name: education, dtype: int64


Были удалены явные дубликаты в датафрейме с помощью метода drop_duplicates() со сбросом индекса методом reset_index()
<br>Не явные дубликаты были удалены  в столбце education приведением всех строк к нижнему регистру методом apply(str.lower)

### Лемматизация

In [7]:
m = Mystem()

text = '' #создал переменную для хранения суммированного значения data['purpose'] каждой строки

for i in data['purpose']:#цикл перебирает каждое значение столбца data['purpose'] и конкатинирует в общий текст и добавляет пробел
    text +=' '
    text+= i
    
lemmas = m.lemmatize(text) # хранения результата функции lemmatize т.е список лемм

print(Counter(lemmas))
# написать вывод по леммам

Counter({' ': 55067, 'недвижимость': 6353, 'покупка': 5900, 'жилье': 4461, 'автомобиль': 4308, 'образование': 4014, 'с': 2918, 'операция': 2604, 'свадьба': 2335, 'свой': 2231, 'на': 2228, 'строительство': 1879, 'высокий': 1374, 'получение': 1315, 'коммерческий': 1312, 'для': 1290, 'жилой': 1231, 'сделка': 941, 'дополнительный': 907, 'заниматься': 904, 'подержать': 853, 'проведение': 773, 'сыграть': 769, 'сдача': 652, 'семья': 638, 'собственный': 635, 'со': 627, 'ремонт': 607, 'приобретение': 461, 'профильный': 436, 'подержанный': 111, '\n': 1})


### Категоризация данных

In [10]:
family_dict = data[['family_status_id','family_status']]
education_dict = data[['education_id','education']] #словари

# нужно удалить дубликаты из словарей
family_dict = family_dict.drop_duplicates().reset_index(drop=True)
education_dict = education_dict.drop_duplicates().reset_index(drop=True)

In [11]:
def income_group(total_income): # категоризация по уровню дохода ( разделил на три категории в зависимости от уровно дохода)
    
    if 0 < total_income <= 80:
        return 'низкий доход'
    elif total_income < 145:
        return 'средний доход'
    else:
        return 'высокий доход'

data['income_group'] = data['total_income'].apply(income_group) # создал новый столбец категорий в зависимости от дохода

In [12]:
def children_group(children): # категоризация по наличию (отсутствию детей)
    
    if children > 0:
        return 'дети есть'
    else:
        return 'детей нет'

data['children_group'] = data['children'].apply(children_group)

In [13]:
def debt_group(debt): # категоризация наличия(отстутвия) задолженности
    if debt:
        return 'имел задолженность'
    else:
        return 'не имел задолженность'

data['debt_group'] = data['debt'].apply(debt_group)

In [14]:
purpose_of_credit = ['ремонт','свадьба','образование','автомобиль','жилье','недвижимость'] # список основных лемм

def purpose_group(purpose):# метод для категоризации целей кредита по часто встречаемым лемам
    for i in purpose_of_credit:
        if i in m.lemmatize(purpose):
            return i
data['purpose_group'] = data['purpose'].apply(purpose_group)
display(data['income_group'].value_counts())

высокий доход    11858
средний доход     7286
низкий доход      2327
Name: income_group, dtype: int64

Выделил несколько словарей из датафрейма, удалил там дубликаты, разделил данные по нескольким категориям, таким как
цели кредита, наличие (отсутвие) задолженности, наличие детей , уровень дохода

##  Результаты

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [15]:
data_pivot_children = data.pivot_table(index='children_group',columns='debt_group',values='debt',aggfunc='count')

data_pivot_children['доля должников от общей группы'] = data_pivot_children.apply(lambda x: round(x[0]/x.sum() *100,1),axis=1)
display(data_pivot_children)

debt_group,имел задолженность,не имел задолженность,доля должников от общей группы
children_group,,,
детей нет,1064,13090,7.5
дети есть,677,6640,9.3


**Вывод**

по данным сводной  таблицы видно что тех у кого нет детей в два раза больше  возвращали кредит в срок в отличии от тех у кого есть дети 

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [16]:
data_pivot_family = data.pivot_table(index='family_status',columns='debt_group',values='debt',aggfunc='count')
data_pivot_family['доля_должников'] = data_pivot_family.apply(lambda x:round( x[0]/x.sum() * 100,1), axis=1)
display(data_pivot_family)

debt_group,имел задолженность,не имел задолженность,доля_должников
family_status,,,
Не женат / не замужем,274,2536,9.8
в разводе,85,1110,7.1
вдовец / вдова,63,896,6.6
гражданский брак,388,3775,9.3
женат / замужем,931,11413,7.5


**Вывод**

<div >
<br>больше всего доля должников в группе людей со статусом не женат не замужем и гражданский брак 
<br> следовательно незначительная взаимосвязь между семейным положением и возвратом кредита в срок присутствует
</div>

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [17]:
data_pivot_income = data.pivot_table(index='income_group',columns='debt_group',values='total_income',aggfunc='count')
data_pivot_income['доля_должников'] = data_pivot_income.apply(lambda x:round( x[0]/x.sum() * 100,1), axis=1)
display(data_pivot_income)


debt_group,имел задолженность,не имел задолженность,доля_должников
income_group,,,
высокий доход,944,10914,8.0
низкий доход,179,2148,7.7
средний доход,618,6668,8.5


**Вывод**

<div>
<br>больше всего доля должников в группе людей со средним доходом 
<br>ниже всего в группелюдей с низким доходом
<br>следовательно люди со средним доходом чаще всего имеют просрочки по кредиту, а люди с низким доходом реже т.е взаимосвязь присутсвтует
</div>

- Как разные цели кредита влияют на его возврат в срок?

In [18]:
data_pivot_purpose = data.pivot_table(index='purpose_group',columns='debt_group',values='purpose',aggfunc='count')
data_pivot_purpose['доля_должников'] = data_pivot_purpose.apply(lambda x:round( x[0]/x.sum() * 100,1), axis=1)
display(data_pivot_purpose)

debt_group,имел задолженность,не имел задолженность,доля_должников
purpose_group,,,
автомобиль,403,3905,9.4
жилье,273,3581,7.1
недвижимость,474,5879,7.5
образование,370,3644,9.2
ремонт,35,572,5.8
свадьба,186,2149,8.0


**Вывод**

<div>

   <br>Больше всего процент должников с целями <b>свадьба, автомобиль,образование</b>
   <br>Следовательно чем серьзнее цель кредита тем меньше процент должников , скорее всего это связано с большими процентами по просрочке кредитов по недвижимости
   <br> Следовательно есть взаимосвязь между целью кредита и возвратом в срок

</div>

## Общий вывод

<div class="alert alert-info">

<br>Был проведен анализ влияния различных категорий на погашения кредита в срок,
также были очищены данне от явных и не явных дубликатов, пропуски в данных были заполнены медианным и средним значением,
проведена лемматизация основных целей кредита, на основе количественных признаком данные были разбиты на категории,
были созданы сводные таблицы на основе которых написаны соответвующие выводы о зависимости данных категорий и возврата кредита в срок.
 <div>